> **Note**
>
> Saved changes to this file will not presist between instances. To retain your edits,
> please save the file to your main directory.



# Running jobs on XQC

## Authentication

Visit https://cloud.xanadu.ai/keys to generate your authentication token. Replace `AUTHENTICATION_TOKEN` below with the generated token and uncomment that line before executing.

In [ ]:
import strawberryfields as sf
# sf.store_account("AUTHENTICATION_TOKEN")
sf.ping()

## Create your program

In [ ]:
import numpy as np
from strawberryfields import ops
from strawberryfields import RemoteEngine

We will calculate the vibronic spectrum of ethylene. The vibronic spectrum represents the wavelengths at which light is most strongly absorbed due to electronic and vibrational excitations. Check out the demo [Quantum algorithms on the Xanadu quantum cloud](https://strawberryfields.ai/photonics/demos/tutorial_X8_demos.html) for more details.

In [ ]:
U = np.array([
    [-0.5349105592386603,  0.8382670887228271,   0.10356058421030308, -0.021311662937477004],
    [-0.6795134137271818, -0.4999083619063322,   0.5369830827402383,   0.001522863010877817],
    [-0.4295084785810517, -0.17320833713971984, -0.7062800928050401,   0.5354341876268153  ],
    [ 0.2601051345260338,  0.13190447151471643,  0.4495473331653913,   0.8443066531962792  ]
])

In [ ]:
prog = sf.Program(8, name="job1")

with prog.context as q:
    # Initial squeezed states
    # Allowed values are r=1.0 or r=0.0
    ops.S2gate(1.0) | (q[0], q[4])
    
    # Interferometer on the signal modes (0-3)
    ops.Interferometer(U) | (q[0], q[1], q[2], q[3])
    ops.Interferometer(U) | (q[4], q[5], q[6], q[7])
    
    ops.MeasureFock() | q

We can use either `"simulon_gaussian"` to run on our remote Gaussian simulator, or `"X8"` to run on our X8 quantum hardware device. Feel free to play around with both (although the simulator might be much slower for many shots).

In [ ]:
eng = RemoteEngine("X8")

In [ ]:
results = eng.run(prog, shots=20)
print(results.samples)

Let's print the mean value of the samples. As well as the number of times no photons were measured at all (i.e., the `[0, 0, 0, 0, 0, 0, 0, 0]` case).

In [ ]:
print(np.mean(results.samples, axis=0))

In [ ]:
from collections import Counter
bitstrings = [tuple(i) for i in results.samples]
counts = {k:v for k, v in Counter(bitstrings).items()}
print(counts[(0, 0, 0, 0, 0, 0, 0, 0)])

## Asynchronous execution

We can also execute the program asynchronously!

In [ ]:
job = eng.run_async(prog, shots=3)

In [ ]:
print(job.id)
print(job.status)

In [ ]:
job.refresh()
print(job.status)

In [ ]:
result = job.result
print(result.samples.shape)

## Hardware run

Finally, we can attempt to plot the vibronic spectrum of ethylene by running with many shots on the hardware, and by using the vibronic module to calculate the absorption energies.

In [ ]:
eng = RemoteEngine("X8")
results = eng.run(prog, shots=400000)
samples = results.samples

In [ ]:
from strawberryfields.apps import vibronic, plot
from matplotlib import pyplot as plt

samples = [list(s) for s in samples]
w = [2979, 1580, 1286, 977]
wp = [2828, 1398, 1227, 855]
energies = vibronic.energies(samples, w, wp)

We can plot the spectra using either `matplotlib` or `plot.ly`.

In [ ]:
plot.spectrum(energies, xmin=-6000, xmax=6000)